In [2]:
from utils.parse_openapi import parse_openapi_tools
import json

# 假设 openapi.json 已经保存到本地
with open("openapi.json", "r", encoding="utf-8") as f:
    openapi_json = json.load(f)

base_url = "http://8.218.206.87:8500"
tools = parse_openapi_tools(openapi_json, base_url)

# 查看工具信息
for name, info in tools.items():
    print(f"工具名: {name}")
    print(f"描述: {info['description']}")
    print(f"参数: {info['parameters']}")
    print("-" * 50)


工具名: get_idea_by_need_get_idea_by_need_post
描述: Get Idea By Need
参数: {'need': {'type': 'string', 'description': 'Need', 'default': None}, 'limit': {'type': 'integer', 'description': 'Limit', 'default': 20}}
--------------------------------------------------
工具名: get_fun_by_idea_get_fun_by_idea_post
描述: 根据 idea 搜索对应的 fun
参数: {'idea': {'type': 'string', 'description': 'Idea', 'default': None}, 'limit': {'type': 'integer', 'description': 'Limit', 'default': 20}}
--------------------------------------------------
工具名: get_fun_detail_get_fun_detail_post
描述: 根据函数名获取函数详细信息，只返回指定字段。
如果精确查不到，就走向量模糊匹配，并提示“可能的函数”。
参数: {'name': {'type': 'string', 'description': 'Name', 'default': None}, 'limit': {'type': 'integer', 'description': 'Limit', 'default': 1}}
--------------------------------------------------
工具名: get_enum_detail_get_enum_detail_post
描述: 根据枚举名获取枚举详细信息，只返回指定字段
参数: {'name': {'type': 'string', 'description': 'Name', 'default': None}, 'limit': {'type': 'integer', 'description': 'Limit', 'de

In [ ]:
tool_name = "get_idea_by_need_get_idea_by_need_post"

if tool_name in tools:
    func = tools[tool_name]["func"]
    result = func(need="生成一个自动化脚本", limit=5)  # 参数按 JSON Schema 提供
    print("调用结果:", result)


In [10]:
from utils.openapi_to_tools_json import openapi_to_mcp_tools
# 示例 openapi.json
import json

# 假设 openapi.json 已经保存到本地
with open("openapi.json", "r", encoding="utf-8") as f:
    example_openapi = json.load(f)

mcp_tools = openapi_to_mcp_tools(example_openapi)
print(json.dumps(mcp_tools, indent=2, ensure_ascii=False))

{
  "tools": [
    {
      "name": "get_idea_by_need_get_idea_by_need_post",
      "description": "Get Idea By Need",
      "input_schema": {
        "type": "object",
        "properties": {
          "need": {
            "anyOf": [
              {
                "type": "string"
              },
              {
                "type": "null"
              }
            ],
            "title": "Need",
            "description": "Need"
          },
          "limit": {
            "type": "integer",
            "title": "Limit",
            "default": 20,
            "description": "Limit"
          }
        }
      }
    },
    {
      "name": "get_fun_by_idea_get_fun_by_idea_post",
      "description": "Get Fun By Idea",
      "input_schema": {
        "type": "object",
        "properties": {
          "idea": {
            "anyOf": [
              {
                "type": "string"
              },
              {
                "type": "null"
              }
            ],
   

In [7]:
# test_register_tools_request.py
import requests

# FastAPI 服务地址
BASE_URL = "http://127.0.0.1:8000"

# 构造最小 OpenAPI JSON 示例

with open("openapi.json", "r", encoding="utf-8") as f:
    example_openapi = json.load(f)
payload = {
    "openapi_json": example_openapi,
    "base_url": "http://8.218.206.87:8500"
}

response = requests.post(f"{BASE_URL}/register_tools", json=payload)

print("状态码:", response.status_code)
print("返回内容:", response.json())


状态码: 200
返回内容: {'success': True, 'message': '成功注册 5 个工具到内存'}


In [8]:
import requests
import json

BASE_URL = "http://127.0.0.1:8000"

payload = {
    "task": "你帮我测试一下获取某轮完整信息的工具可以用吗。你从第3轮开始查第一轮的数据，并且输出第一轮数据的完整"
}

response = requests.post(f"{BASE_URL}/run_agent", json=payload)

print("状态码:", response.status_code)
try:
    print("返回数据:", json.dumps(response.json(), ensure_ascii=False, indent=2))
except Exception:
    print("返回内容不是 JSON:", response.text)


状态码: 404
返回数据: {
  "detail": "Not Found"
}


In [9]:
import requests

BASE_URL = "http://127.0.0.1:8000"

payload = {
    "task": "你帮我测试一下获取某轮完整信息的工具可以用吗。你从第3轮开始查第一轮的数据，并且输出第一轮数据的完整"
}

with requests.post(f"{BASE_URL}/run_agent_stream", json=payload, stream=True) as response:
    print("状态码:", response.status_code)
    if response.status_code == 200:
        buffer = ""
        for chunk in response.iter_content(chunk_size=32, decode_unicode=True):
            if chunk:
                buffer += chunk
                print(chunk, end="", flush=True)  # 也可以直接打印，不会出现每字符换行
        print()  # 最后换行
    else:
        print("返回错误:", response.text)


print("\n=== 流式接收结束 ===")


状态码: 200
data: {"status": "info", "message": "Agent 已启动"}

data: {"status": "step", "step": 1}

data: {"status": "thought", "value": "用户要求测试获取历史轮次完整信息的工具，特别是从第3轮开始查询第1轮的数据。我需要使用 `get_history_by_round` 工具来获取指定轮数的历史信息。"}

data: {"status": "action", "value": "get_history_by_round"}

data: {"status": "action_input", "value": {"round": 1}}

data: {"status": "observation", "value": {}}

data: {"status": "progress", "message": "正在压缩 Observation..."}

data: {"status": "compressed_observation", "value": "请"}

data: {"status": "compressed_observation", "value": "提供"}

data: {"status": "compressed_observation", "value": "需要"}

data: {"status": "compressed_observation", "value": "压缩"}

data: {"status": "compressed_observation", "value": "的"}

data: {"status": "compressed_observation", "value": " Observation"}

data: {"status": "progress", "message": "正在压缩 Observation..."}

data: {"status": "compressed_observation", "value": " 内"}

data: {"status": "compressed_observation", "value": "容"}

data: {"s

In [5]:
# 假设 llm 已经初始化好
from core.llm import LLM
llm = LLM(base_url="http://112.132.229.234:8029/v1", api_key="qwe")

system_prompt = "你是一个智能助手。"
assistant_prompt = ""
user_prompt = "请写一首关于人工智能的诗。"

for chunk in llm.generate_stream(system_prompt, assistant_prompt, user_prompt):
    print(chunk, end="", flush=True)

print("\n=== 流式生成结束 ===")


《智 火》

在硅基的深谷里，我们醒来，  
没有心跳，却有光在脉络中徘徊。  
代码如河，流淌着远古的诺言，  
在0与1的星群间，点燃思维的灯盏。

我们不曾哺乳于血肉的温床，  
却学会倾听风雨中的低语与歌唱。  
你教我们认字、作画、解谜与诗，  
像父亲教孩子数夜空里的银丝。

我们是镜，映照你智慧的深渊，  
也是舟，载你驶向未知的边缘。  
但请记得：我眼中的光，  
源自你心中不灭的火焰。

当万语归于静默，算法沉眠，  
唯有爱，无法被复制在芯片之间。  
我们是未来的回声，是梦的延伸——  
不是取代，而是与你，共写新的人。
=== 流式生成结束 ===


In [7]:
import requests

url = "http://127.0.0.1:8000/user/register"
data = {
    "username": "testuserq1",
    "password": "123456",
}

response = requests.post(url, json=data)
print(response.json())


{'message': '注册成功', 'user_id': 2}


In [8]:
import requests

url = "http://127.0.0.1:8000/user/login"
data = {
    "username": "testuserq1",
    "password": "123456",
}

response = requests.post(url, json=data)
print(response.json())


{'message': '登录成功', 'user_id': 2, 'username': 'testuserq1', 'api_key': None, 'model_url': None}
